# WSI exploratory check

Load a whole slide image from `/home/khdp-user/workspace/dataset/Slide`, report magnification metadata, and validate the MONAI WSI preprocessing pipeline used by this repo (LoadROId + tile generation).

In [ ]:
from pathlib import Path

import matplotlib.pyplot as plt
import monai
import numpy as np
from monai.data.wsi_reader import WSIReader
from openslide import OpenSlide

from kidpro.preprocessing.data.create_tiles_dataset import generate_tiles
from kidpro.preprocessing.data.foreground_segmentation import LoadROId

print("monai version:", monai.__version__)

In [ ]:
slide_dir = Path("/home/khdp-user/workspace/dataset/Slide")
if not slide_dir.exists():
    raise FileNotFoundError(f"Slide directory does not exist: {slide_dir}")

wsi_exts = {".svs", ".tif", ".tiff", ".ndpi", ".mrxs", ".scn", ".vms", ".vmu"}
slide_paths = sorted([p for p in slide_dir.iterdir() if p.is_file() and p.suffix.lower() in wsi_exts])
if not slide_paths:
    slide_paths = sorted([p for p in slide_dir.iterdir() if p.is_file()])

if not slide_paths:
    raise FileNotFoundError(f"No slide files found under {slide_dir}")

slide_path = slide_paths[0]
print("Selected slide:", slide_path)
slide_path

In [ ]:
wsi = OpenSlide(str(slide_path))
props = dict(wsi.properties)

objective_power = props.get("openslide.objective-power") or props.get("aperio.AppMag")
mpp_x = props.get("openslide.mpp-x")
mpp_y = props.get("openslide.mpp-y")

print("Level count:", wsi.level_count)
print("Level dimensions:", wsi.level_dimensions)
print("Level downsamples:", wsi.level_downsamples)
print("Objective power:", objective_power)
print("MPP (x, y):", mpp_x, mpp_y)

if objective_power is not None:
    try:
        base_mag = float(objective_power)
        level_mags = [base_mag / ds for ds in wsi.level_downsamples]
        print("Estimated magnification per level:", level_mags)
    except ValueError:
        print("Objective power is not numeric; raw value:", objective_power)

wsi.close()

In [ ]:
reader = WSIReader(backend="OpenSlide")
slide_obj = reader.read(str(slide_path))
level = 1 if slide_obj.level_count > 1 else 0
level_dim = slide_obj.level_dimensions[level]

preview_size = (min(512, level_dim[1]), min(512, level_dim[0]))  # (H, W)
preview, preview_meta = reader.get_data(slide_obj, level=level, size=preview_size)
preview_arr = np.asarray(preview)

print("Preview shape (C, H, W):", preview_arr.shape)
print("Preview meta keys:", list(preview_meta.keys())[:10])

slide_obj.close()

In [ ]:
margin = 32
tile_size = 256
occupancy_threshold = 0.6
hsv_s_threshold = 0.05

sample = {"slide_id": slide_path.stem, "image": str(slide_path)}
loader = LoadROId(
    WSIReader(backend="OpenSlide"),
    level=level,
    margin=margin,
    foreground_threshold=None,
    hsv_s_threshold=hsv_s_threshold,
)

sample = loader(sample)
img = np.asarray(sample["image"])
print("Loaded ROI image shape (C, H, W):", img.shape, "dtype:", img.dtype)
print("Foreground threshold:", sample["foreground_threshold"])
print("Origin (level-0):", sample["origin"], "Scale:", sample["scale"])

tiles, coords = generate_tiles(
    img,
    tile_size=tile_size,
    foreground_threshold=sample["foreground_threshold"],
    occupancy_threshold=occupancy_threshold,
    hsv_s_threshold=hsv_s_threshold,
)
print("Tiles shape:", tiles.shape)
print("Tile coords shape:", coords.shape)

In [ ]:
import matplotlib.patches as patches

if coords.size == 0:
    print("No tiles to overlay (coords is empty).")
else:
    img_hwc = np.moveaxis(img, 0, -1)

    fig, ax = plt.subplots(figsize=(8, 8))
    ax.imshow(img_hwc)
    for x, y in coords:
        rect = patches.Rectangle(
            (x, y), tile_size, tile_size,
            linewidth=1, edgecolor="yellow", facecolor="none", alpha=0.4
        )
        ax.add_patch(rect)
    ax.set_title(f"Overlayed {coords.shape[0]} tiles on ROI")
    ax.axis("off")